# Assignment 1

You only need to write one line of code for each question. When answering questions that ask you to identify or interpret something, the length of your response doesn’t matter. For example, if the answer is just ‘yes,’ ‘no,’ or a number, you can just give that answer without adding anything else.

We will go through comparable code and concepts in the live learning session. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that **no outside searches are required by the assignment!**). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

### Classification using KNN

Let's set up our workspace and use the **Wine dataset** from `scikit-learn`. This dataset contains 178 wine samples with 13 chemical features, used to classify wines into different classes based on their origin.

The **response variable** is `class`, which indicates the type of wine. We'll use all of the chemical features to predict this response variable.

In [360]:
# Import standard libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [361]:
from sklearn.datasets import load_wine

# Load the Wine dataset
wine_data = load_wine()

# Convert to DataFrame
wine_df = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)

# Bind the 'class' (wine target) to the DataFrame
wine_df['class'] = wine_data.target

# Display the DataFrame
wine_df


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


#### **Question 1:** 
#### Data inspection

Before fitting any model, it is essential to understand our data. **Use Python code** to answer the following questions about the **Wine dataset**:

_(i)_ How many observations (rows) does the dataset contain?

In [362]:
# Your answer here 

num_rows = wine_df.shape[0]
print("number of rows:", num_rows)



number of rows: 178


In [363]:
num_column = wine_df.shape[1]
print("number of column:", num_column)


number of column: 14


_(ii)_ How many variables (columns) does the dataset contain?

In [364]:
# Your answer here
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
 13  class

_(iii)_ What is the 'variable type' of the response variable `class` (e.g., 'integer', 'category', etc.)? What are the 'levels' (unique values) of the variable?

In [365]:
# Your answer here
wine_df['class'].unique()


array([0, 1, 2])


_(iv)_ How many predictor variables do we have (Hint: all variables other than `class`)? 

In [366]:
# Your answer here
num_predictors = wine_df.drop(columns=['class']).shape[1]
print ('predictor variables', num_predictors)



predictor variables 13


You can use `print()` and `describe()` to help answer these questions.

#### **Question 2:** 
#### Standardization and data-splitting

Next, we must preform 'pre-processing' or 'data munging', to prepare our data for classification/prediction. For KNN, there are three essential steps. A first essential step is to 'standardize' the predictor variables. We can achieve this using the scaler method, provided as follows:

In [367]:
# Select predictors (excluding the last column)
predictors = wine_df.iloc[:, :-1]

# Standardize the predictors
scaler = StandardScaler()
predictors_standardized = pd.DataFrame(scaler.fit_transform(predictors), columns=predictors.columns)

# Display the head of the standardized predictors
print(predictors_standardized.head())

    alcohol  malic_acid  ...  od280/od315_of_diluted_wines   proline
0  1.518613   -0.562250  ...                      1.847920  1.013009
1  0.246290   -0.499413  ...                      1.113449  0.965242
2  0.196879    0.021231  ...                      0.788587  1.395148
3  1.691550   -0.346811  ...                      1.184071  2.334574
4  0.295700    0.227694  ...                      0.449601 -0.037874

[5 rows x 13 columns]


(i) Why is it important to standardize the predictor variables?

> Your answer here...#
Measure random variable with different means and variance on a single scale without any complication. It is important to standardize the predictor variable to ensure fair contribution from all variables. Standardization allowing model to consider each variable equally bases on there relationship.

(ii) Why did we elect not to standard our response variable `Class`?

> Your answer here...response variable is nominal categories , not numerical quantities. no need to scale up.

(iii) A second essential step is to set a random seed. Do so below (Hint: use the random.seed function). Why is setting a seed important? Is the particular seed value important? Why or why not?

> your answer here...
Seed is essential to control randomness. Seed value to ensure that sequence random number are reproducible. setting same seed value is impotant to get same result each time. Different seed value result in differnt random patterns, but same value ensure same result.

(iv) A third essential step is to split our standardized data into separate training and testing sets. We will split into 75% training and 25% testing. The provided code randomly partitions our data, and creates linked training sets for the predictors and response variables. 

Extend the code to create a non-overlapping test set for the predictors and response variables.

In [368]:
# set a seed for reproducibility
np.random.seed(1)

# Your code here ...
#split our data
wine_df_train, wine_df_test = train_test_split(
predictors_standardized, train_size = 0.75, shuffle= True, stratify = wine_df["class"]
)

In [369]:
wine_df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 133 entries, 157 to 143
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       133 non-null    float64
 1   malic_acid                    133 non-null    float64
 2   ash                           133 non-null    float64
 3   alcalinity_of_ash             133 non-null    float64
 4   magnesium                     133 non-null    float64
 5   total_phenols                 133 non-null    float64
 6   flavanoids                    133 non-null    float64
 7   nonflavanoid_phenols          133 non-null    float64
 8   proanthocyanins               133 non-null    float64
 9   color_intensity               133 non-null    float64
 10  hue                           133 non-null    float64
 11  od280/od315_of_diluted_wines  133 non-null    float64
 12  proline                       133 non-null    float64
dtypes: float

In [370]:
wine_df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45 entries, 43 to 125
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       45 non-null     float64
 1   malic_acid                    45 non-null     float64
 2   ash                           45 non-null     float64
 3   alcalinity_of_ash             45 non-null     float64
 4   magnesium                     45 non-null     float64
 5   total_phenols                 45 non-null     float64
 6   flavanoids                    45 non-null     float64
 7   nonflavanoid_phenols          45 non-null     float64
 8   proanthocyanins               45 non-null     float64
 9   color_intensity               45 non-null     float64
 10  hue                           45 non-null     float64
 11  od280/od315_of_diluted_wines  45 non-null     float64
 12  proline                       45 non-null     float64
dtypes: float64

#### **Question 3:**
#### Model initialization and cross-validation
We are finally set to fit the KNN model. 


Perform a grid search to tune the `n_neighbors` hyperparameter using 10-fold cross-validation. Follow these steps:

1. Initialize the KNN classifier using `KNeighborsClassifier()`.
2. Define a parameter grid for `n_neighbors` ranging from 1 to 50.
3. Implement a grid search using `GridSearchCV` with 10-fold cross-validation to find the optimal number of neighbors.
4. After fitting the model on the training data, identify and return the best value for `n_neighbors` based on the grid search results.

In [403]:
X = wine_df_train
y = wine_df_test


In [404]:
wine_df_validation = train_test_split(wine_df_train, train_size = 0.75, shuffle = True) #, stratify=wine_df_train["class"])


In [405]:
# Your code here...
knn = KNeighborsClassifier()
knn

KNeighborsClassifier()

In [375]:
columns = wine_df_train.columns #.columns.drop("class")
columns

Index(['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium',
       'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue',
       'od280/od315_of_diluted_wines', 'proline'],
      dtype='object')

In [395]:

#cross-validation
knn = KNeighborsClassifier()
X = wine_df.drop(columns=['class'])
y = wine_df['class']


In [399]:
returned_dictionary = cross_validate(
   estimator = knn ,
   cv = 10,
   X = X,
   y = y
)

In [401]:
cv_10_df = pd.DataFrame(returned_dictionary)
cv_10_df

,fit_time,score_time,test_score
0,0.001999,0.003584,0.666667
1,0.002565,0.003114,0.666667
2,0.003076,0.003026,0.611111
3,0.001993,0.003000,0.611111
4,0.002037,0.004107,0.611111
5,0.002002,0.003063,0.611111
6,0.001998,0.002006,0.722222
7,0.001025,0.002998,0.666667
8,0.002090,0.002981,0.823529
9,0.002672,0.003049,0.764706


In [402]:
#which values of k we want to evaluate
parmeter_grid = {
    "n_neighbors": range(1, 51)
}

In [407]:
wine_df_tune_grid = GridSearchCV(
    estimator = knn,
    param_grid = parmeter_grid,
    cv = 10
)

In [412]:
wine_df_tune_grid.fit (wine_df.drop(columns=['class']),
wine_df["class"])

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 51)})

#### **Question 4:**
#### Model evaluation

Using the best value for `n_neighbors`, fit a KNN model on the training data and evaluate its performance on the test set using `accuracy_score`.

In [ ]:

best_knn = KNeighborsClassifier()
best_knn.fit (wine_df.drop(columns=['class']),
wine_df["class"])

KNeighborsClassifier()

In [426]:
accuracy_grid = pd.DataFrame(wine_df_tune_grid.cv_results_)
accuracy_grid

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002321,0.000734,0.003801,0.000933,1,{'n_neighbors': 1},0.833333,0.777778,0.666667,0.555556,0.611111,0.833333,0.777778,0.777778,0.823529,0.823529,0.748039,0.095405,1
1,0.003015,0.000795,0.004758,0.001175,2,{'n_neighbors': 2},0.555556,0.666667,0.555556,0.611111,0.611111,0.666667,0.777778,0.666667,0.823529,0.764706,0.669935,0.087932,48
2,0.001808,0.000403,0.002810,0.000566,3,{'n_neighbors': 3},0.666667,0.611111,0.666667,0.666667,0.666667,0.666667,0.722222,0.777778,0.882353,0.882353,0.720915,0.090626,4
3,0.001847,0.000695,0.003106,0.000499,4,{'n_neighbors': 4},0.666667,0.666667,0.611111,0.611111,0.611111,0.611111,0.611111,0.611111,0.823529,0.823529,0.664706,0.082275,50
4,0.001779,0.000427,0.003140,0.000208,5,{'n_neighbors': 5},0.666667,0.666667,0.611111,0.611111,0.611111,0.611111,0.722222,0.666667,0.823529,0.764706,0.675490,0.069908,46
5,0.001936,0.000438,0.003323,0.000583,6,{'n_neighbors': 6},0.666667,0.722222,0.611111,0.555556,0.666667,0.611111,0.666667,0.555556,0.823529,0.823529,0.670261,0.091123,47
6,0.001838,0.000498,0.003162,0.000516,7,{'n_neighbors': 7},0.666667,0.722222,0.611111,0.500000,0.611111,0.611111,0.722222,0.722222,0.823529,0.705882,0.669608,0.085061,49
7,0.002025,0.000629,0.003205,0.000246,8,{'n_neighbors': 8},0.722222,0.722222,0.666667,0.666667,0.611111,0.666667,0.833333,0.611111,0.823529,0.764706,0.708824,0.075212,20
8,0.001936,0.000563,0.002964,0.000583,9,{'n_neighbors': 9},0.722222,0.722222,0.666667,0.611111,0.611111,0.611111,0.722222,0.722222,0.764706,0.823529,0.697712,0.067959,32
9,0.002293,0.000389,0.002732,0.000527,10,{'n_neighbors': 10},0.722222,0.666667,0.611111,0.777778,0.611111,0.666667,0.722222,0.666667,0.705882,0.823529,0.697386,0.064342,39


# Criteria


| **Criteria**                                           | **Complete**                                      | **Incomplete**                                    |
|--------------------------------------------------------|---------------------------------------------------|--------------------------------------------------|
| **Data Inspection**                                    | Data is inspected for number of variables, observations and data types. | Data inspection is missing or incomplete.         |
| **Data Scaling**                                       | Data scaling or normalization is applied where necessary (e.g., using `StandardScaler`). | Data scaling or normalization is missing or incorrectly applied. |
| **Model Initialization**                               | The KNN model is correctly initialized and a random seed is set for reproducibility.            | The KNN model is not initialized, is incorrect, or lacks a random seed for reproducibility. |
| **Parameter Grid for `n_neighbors`**                   | The parameter grid for `n_neighbors` is correctly defined. | The parameter grid is missing or incorrectly defined. |
| **Cross-Validation Setup**                             | Cross-validation is set up correctly with 10 folds. | Cross-validation is missing or incorrectly set up. |
| **Best Hyperparameter (`n_neighbors`) Selection**       | The best value for `n_neighbors` is identified using the grid search results. | The best `n_neighbors` is not selected or incorrect. |
| **Model Evaluation on Test Data**                      | The model is evaluated on the test data using accuracy. | The model evaluation is missing or uses the wrong metric. |


## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Note:

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

### Submission Parameters:
* Submission Due Date: `11:59 PM - 09/07/2025`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/LCR/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-7-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
